[View in Colaboratory](https://colab.research.google.com/github/thundercat95/AutoML/blob/master/SMAC%20preprocessor.ipynb)

## Installation

In [0]:
!pip install openml
!apt-get install build-essential swig
!pip install smac
!pip install xgboost
!apt-get install vim

## Imports

In [0]:
import numpy as np
import pandas as pd
import time
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from sklearn import decomposition
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from xgboost import XGBClassifier
import openml as oml
apikey = '2f6c58f4f6d62ea93dd32764ec88089c'
oml.config.apikey = apikey

# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, UniformIntegerHyperparameter
from ConfigSpace.conditions import *
from ConfigSpace import Configuration

# Import SMAC-utilities
from smac.tae.execute_func import ExecuteTAFuncDict
from smac.scenario.scenario import Scenario
from smac.facade.smac_facade import SMAC

## Test with Splice Dataset (OpenML id : 46)

In [0]:
# We load Splice dataset from openml
splice = oml.datasets.get_dataset(46)

#Preprocessor normalize hyper parameter
norm = CategoricalHyperparameter("norm", ["l1", "l2", "max"], default_value="l2")

# Build Configuration Space for Model
cs = ConfigurationSpace()
max_depth = UniformIntegerHyperparameter("max_depth", 1, 10, default_value=3)
learning_rate = UniformFloatHyperparameter("learning_rate", 0.001, 1.0, default_value=0.1)
n_estimators = UniformIntegerHyperparameter("n_estimators", 10, 500, default_value=100)
gamma = UniformFloatHyperparameter("gamma", 0.0, 1.0, default_value=0.0)
min_child_weight = UniformIntegerHyperparameter("min_child_weight", 1, 50, default_value=1)
max_delta_step  = UniformIntegerHyperparameter("max_delta_step ", 0, 50, default_value=0)
subsample = UniformFloatHyperparameter("subsample", 0.0, 1.0, default_value=1.0)
cs.add_hyperparameters([max_depth, learning_rate, n_estimators, gamma, min_child_weight, max_delta_step, subsample,norm])


def model_with_pre(cfg):
    print(cfg)
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    pre_cfg ={}
    X, y = splice.get_data(target=splice.default_target_attribute)
    pre_cfg['norm'] = cfg['norm']
    print(pre_cfg)
    pre_cfg = {k : pre_cfg[k] for k in pre_cfg if pre_cfg[k]}
    X_new = normalize(X, **pre_cfg)
    del cfg['norm']
    xgbc = XGBClassifier(**cfg, random_state=42)
    scores = cross_val_score(xgbc, X_new, y, cv=5)
    print(np.mean(scores))
    return 1-np.mean(scores)  # Minimize!
  
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 5,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })

# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
        tae_runner=model_with_pre)
start = time.time()
incumbent = smac.optimize()
inc_value = model_with_pre(incumbent)
end = time.time()
print("Optimized Value: %.5f" % (1-inc_value))
print("Best parameter : ", incumbent.get_dictionary())
print("Time required in seconds :", (end - start))




Optimizing! Depending on your machine, this might take a few minutes.
Configuration:
  gamma, Value: 0.0
  learning_rate, Value: 0.1
  max_delta_step , Value: 0
  max_depth, Value: 3
  min_child_weight, Value: 1
  n_estimators, Value: 100
  norm, Value: 'l2'
  subsample, Value: 1.0

{'norm': 'l2'}


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9564272318105663
Configuration:
  gamma, Value: 0.7359525285652845
  learning_rate, Value: 0.8117657600950038
  max_delta_step , Value: 48
  max_depth, Value: 5
  min_child_weight, Value: 12
  n_estimators, Value: 103
  norm, Value: 'max'
  subsample, Value: 0.46610295102568133

{'norm': 'max'}


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9448260081924135
Configuration:
  gamma, Value: 0.6108620947042035
  learning_rate, Value: 0.8363471290456959
  max_delta_step , Value: 5
  max_depth, Value: 7
  min_child_weight, Value: 23
  n_estimators, Value: 68
  norm, Value: 'max'
  subsample, Value: 0.18550419634224635

{'norm': 'max'}


...

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9564301799020456
Configuration:
  gamma, Value: 0.9187416722392869
  learning_rate, Value: 0.6433665259425442
  max_delta_step , Value: 11
  max_depth, Value: 6
  min_child_weight, Value: 4
  n_estimators, Value: 434
  norm, Value: 'l2'
  subsample, Value: 0.5109722934233317

{'norm': 'l2'}


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9564301799020456
Optimized Value: 0.95643
Best parameter :  {'gamma': 0.9187416722392869, 'learning_rate': 0.6433665259425442, 'max_delta_step ': 11, 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 434, 'norm': 'l2', 'subsample': 0.5109722934233317}
Time required in seconds : 136.17632913589478


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Pipeline

In [0]:
# We load Splice dataset from openml
splice = oml.datasets.get_dataset(46)

#Preprocessor normalize hyper parameter
norm = CategoricalHyperparameter("norm", ["l1", "l2", "max"], default_value="l2")

# Build Configuration Space for Model
cs = ConfigurationSpace()
max_depth = UniformIntegerHyperparameter("max_depth", 1, 10, default_value=3)
learning_rate = UniformFloatHyperparameter("learning_rate", 0.001, 1.0, default_value=0.1)
n_estimators = UniformIntegerHyperparameter("n_estimators", 10, 500, default_value=100)
gamma = UniformFloatHyperparameter("gamma", 0.0, 1.0, default_value=0.0)
min_child_weight = UniformIntegerHyperparameter("min_child_weight", 1, 50, default_value=1)
max_delta_step  = UniformIntegerHyperparameter("max_delta_step ", 0, 50, default_value=0)
subsample = UniformFloatHyperparameter("subsample", 0.0, 1.0, default_value=1.0)
cs.add_hyperparameters([max_depth, learning_rate, n_estimators, gamma, min_child_weight, max_delta_step, subsample,norm])


def model_with_pre(cfg):
    print(cfg)
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    pre_cfg ={}
    X, y = splice.get_data(target=splice.default_target_attribute)
    pre_cfg['norm'] = cfg['norm']
    print(pre_cfg)
    pre_cfg = {k : pre_cfg[k] for k in pre_cfg if pre_cfg[k]}
    del cfg['norm']
    print(norm)
    final = Pipeline([('normalize', Normalizer(**pre_cfg)), ('Xgboost', XGBClassifier(**cfg))])
    scores = cross_val_score(final, X, y, cv=5)
    print(np.mean(scores))
    return 1-np.mean(scores)  # Minimize!
  
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 5,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })

# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
        tae_runner=model_with_pre)
start = time.time()
incumbent = smac.optimize()
inc_value = model_with_pre(incumbent)
end = time.time()
print("Optimized Value: %.5f" % (1-inc_value))  
print("Best parameter : ", incumbent.get_dictionary())
print("Time required in seconds :", (end - start))

Optimizing! Depending on your machine, this might take a few minutes.
Configuration:
  gamma, Value: 0.0
  learning_rate, Value: 0.1
  max_delta_step , Value: 0
  max_depth, Value: 3
  min_child_weight, Value: 1
  n_estimators, Value: 100
  norm, Value: 'l2'
  subsample, Value: 1.0

{'norm': 'l2'}
norm, Type: Categorical, Choices: {l1, l2, max}, Default: l2


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9564272318105663
Configuration:
  gamma, Value: 0.6326156411092102
  learning_rate, Value: 0.9931395961792671
  max_delta_step , Value: 50
  max_depth, Value: 3
  min_child_weight, Value: 36
  n_estimators, Value: 135
  norm, Value: 'l2'
  subsample, Value: 0.4570085158407664

{'norm': 'l2'}
norm, Type: Categorical, Choices: {l1, l2, max}, Default: l2


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9366843622238058
Configuration:
  gamma, Value: 0.6108620947042035
  learning_rate, Value: 0.8363471290456959
  max_delta_step , Value: 5
  max_depth, Value: 7
  min_child_weight, Value: 23
  n_estimators, Value: 68
  norm, Value: 'max'
  subsample, Value: 0.18550419634224635

{'norm': 'max'}
norm, Type: Categorical, Choices: {l1, l2, max}, Default: l2


...

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9529818963385883
Configuration:
  gamma, Value: 0.0
  learning_rate, Value: 0.1
  max_delta_step , Value: 0
  max_depth, Value: 3
  min_child_weight, Value: 1
  n_estimators, Value: 100
  norm, Value: 'l2'
  subsample, Value: 1.0

{'norm': 'l2'}
norm, Type: Categorical, Choices: {l1, l2, max}, Default: l2


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9564272318105663
Optimized Value: 0.95643
Best parameter :  {'gamma': 0.0, 'learning_rate': 0.1, 'max_delta_step ': 0, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'norm': 'l2', 'subsample': 1.0}
Time required in seconds : 98.66317224502563


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## JSON

### SVM

In [20]:
import json
from ConfigSpace.read_and_write.json import write, read

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

# We define a few possible types of SVM-kernels and add them as "kernel" to our cs
kernel = CategoricalHyperparameter("kernel", ["linear", "rbf", "poly", "sigmoid"], default_value="poly")
cs.add_hyperparameter(kernel)

# There are some hyperparameters shared by all kernels
C = UniformFloatHyperparameter("C", 0.001, 1000.0, default_value=1.0)
shrinking = CategoricalHyperparameter("shrinking", ["true", "false"], default_value="true")
cs.add_hyperparameters([C, shrinking])

# Others are kernel-specific, so we can add conditions to limit the searchspace
degree = UniformIntegerHyperparameter("degree", 1, 5, default_value=3)     # Only used by kernel poly
coef0 = UniformFloatHyperparameter("coef0", 0.0, 10.0, default_value=0.0)  # poly, sigmoid
cs.add_hyperparameters([degree, coef0])
use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef0 = InCondition(child=coef0, parent=kernel, values=["poly", "sigmoid"])
cs.add_conditions([use_degree, use_coef0])

# This also works for parameters that are a mix of categorical and values from a range of numbers
# For example, gamma can be either "auto" or a fixed float
gamma = CategoricalHyperparameter("gamma", ["auto", "value"], default_value="auto")  # only rbf, poly, sigmoid
gamma_value = UniformFloatHyperparameter("gamma_value", 0.0001, 8, default_value=1)
cs.add_hyperparameters([gamma, gamma_value])

# We only activate gamma_value if gamma is set to "value"
cs.add_condition(InCondition(child=gamma_value, parent=gamma, values=["value"]))
# And again we can restrict the use of gamma in general to the choice of the kernel
cs.add_condition(InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"]))

z = write(cs)
list_z = json.loads(z)

print(z)
print(list_z["hyperparameters"][0]["name"])

with open('data.json', 'w') as outfile:
    json.dump(list_z, outfile)
    
with open('data.json', 'r') as f:
  datastore = json.load(f)
  
print(datastore)


{
  "hyperparameters": [
    {
      "name": "C",
      "type": "uniform_float",
      "log": false,
      "lower": 0.001,
      "upper": 1000.0,
      "default": 1.0
    },
    {
      "name": "kernel",
      "type": "categorical",
      "choices": [
        "linear",
        "rbf",
        "poly",
        "sigmoid"
      ],
      "default": "poly"
    },
    {
      "name": "shrinking",
      "type": "categorical",
      "choices": [
        "true",
        "false"
      ],
      "default": "true"
    },
    {
      "name": "coef0",
      "type": "uniform_float",
      "log": false,
      "lower": 0.0,
      "upper": 10.0,
      "default": 0.0
    },
    {
      "name": "degree",
      "type": "uniform_int",
      "log": false,
      "lower": 1,
      "upper": 5,
      "default": 3
    },
    {
      "name": "gamma",
      "type": "categorical",
      "choices": [
        "auto",
        "value"
      ],
      "default": "auto"
    },
    {
      "name": "gamma_value",
      "type": 

In [24]:
type(0.5)

float

## Test

In [0]:
b = dict()
b['C'] = 0.1
BaseEstimator.get_params(LogisticRegression(),deep=True)

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [0]:
final = Pipeline([('normalize', Normalizer()), ('Xgboost', XGBClassifier())])
z = final.steps[1][1]
cs1 = ConfigurationSpace()
cs2 = ConfigurationSpace()
cs3 = ConfigurationSpace()
max_depth = UniformIntegerHyperparameter("max_depth", 1, 10, default_value=3)
cs1.add_hyperparameters([max_depth])
cs1.get_all_unconditional_hyperparameters()

['max_depth']

In [0]:
# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

# We define a few possible types of SVM-kernels and add them as "kernel" to our cs
kernel = CategoricalHyperparameter("kernel", ["linear", "rbf", "poly", "sigmoid"], default_value="poly")
cs.add_hyperparameter(kernel)

# There are some hyperparameters shared by all kernels
C = UniformFloatHyperparameter("C", 0.001, 1000.0, default_value=1.0)
shrinking = CategoricalHyperparameter("shrinking", ["true", "false"], default_value="true")
cs.add_hyperparameters([C, shrinking])

# Others are kernel-specific, so we can add conditions to limit the searchspace
degree = UniformIntegerHyperparameter("degree", 1, 5, default_value=3)     # Only used by kernel poly
coef0 = UniformFloatHyperparameter("coef0", 0.0, 10.0, default_value=0.0)  # poly, sigmoid
cs.add_hyperparameters([degree, coef0])
use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef0 = InCondition(child=coef0, parent=kernel, values=["poly", "sigmoid"])
cs.add_conditions([use_degree, use_coef0])

# This also works for parameters that are a mix of categorical and values from a range of numbers
# For example, gamma can be either "auto" or a fixed float
gamma = CategoricalHyperparameter("gamma", ["auto", "value"], default_value="auto")  # only rbf, poly, sigmoid
gamma_value = UniformFloatHyperparameter("gamma_value", 0.0001, 8, default_value=1)
cs.add_hyperparameters([gamma, gamma_value])

# We only activate gamma_value if gamma is set to "value"
cs.add_condition(InCondition(child=gamma_value, parent=gamma, values=["value"]))
# And again we can restrict the use of gamma in general to the choice of the kernel
cs.add_condition(InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"]))

def convert_cs_to_dictionary(cfg, num):
  cfg_dict = dict()
  list_unconditional = cfg.get_all_unconditional_hyperparameters()
  list_confitional = cfg.get_all_conditional_hyperparameters()
  list_all = list_unconditional + list_confitional
  for hyperparameter in list_all:
    cfg_dict[""]

cs_to_dict = convert_cs_to_dictionary(cs)

gamma | kernel in {'rbf', 'poly', 'sigmoid'}

In [21]:
final = Pipeline([('normalize', Normalizer()), ('Xgboost', XGBClassifier())])

#Preprocessor normalize hyper parameter
norm = CategoricalHyperparameter("norm", ["l1", "l2", "max"], default_value="l2")
cs2 = ConfigurationSpace()
cs2.add_hyperparameter(norm)

# Build Configuration Space for Model
cs1 = ConfigurationSpace()
max_depth = UniformIntegerHyperparameter("max_depth", 1, 10, default_value=3)
learning_rate = UniformFloatHyperparameter("learning_rate", 0.001, 1.0, default_value=0.1)
n_estimators = UniformIntegerHyperparameter("n_estimators", 10, 500, default_value=100)
gamma = UniformFloatHyperparameter("gamma", 0.0, 1.0, default_value=0.0)
min_child_weight = UniformIntegerHyperparameter("min_child_weight", 1, 50, default_value=1)
max_delta_step  = UniformIntegerHyperparameter("max_delta_step ", 0, 50, default_value=0)
subsample = UniformFloatHyperparameter("subsample", 0.0, 1.0, default_value=1.0)
cs1.add_hyperparameters([max_depth, learning_rate, n_estimators, gamma, min_child_weight, max_delta_step, subsample])



[max_depth, Type: UniformInteger, Range: [1, 10], Default: 3,
 learning_rate, Type: UniformFloat, Range: [0.001, 1.0], Default: 0.1,
 n_estimators, Type: UniformInteger, Range: [10, 500], Default: 100,
 gamma, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.0,
 min_child_weight, Type: UniformInteger, Range: [1, 50], Default: 1,
 max_delta_step , Type: UniformInteger, Range: [0, 50], Default: 0,
 subsample, Type: UniformFloat, Range: [0.0, 1.0], Default: 1.0]

In [0]:
def get_parameter_of_pipeline(node_name) : 
  if node_name == "normalize" :
    norm = CategoricalHyperparameter("norm", ["l1", "l2", "max"], default_value="l2")
    cs2 = ConfigurationSpace()
    cs2.add_hyperparameter(norm)
    return cs2
  if node_name == "Xgboost" : 
    cs1 = ConfigurationSpace()
    max_depth = UniformIntegerHyperparameter("max_depth", 1, 10, default_value=3)
    learning_rate = UniformFloatHyperparameter("learning_rate", 0.001, 1.0, default_value=0.1)
    n_estimators = UniformIntegerHyperparameter("n_estimators", 10, 500, default_value=100)
    gamma = UniformFloatHyperparameter("gamma", 0.0, 1.0, default_value=0.0)
    min_child_weight = UniformIntegerHyperparameter("min_child_weight", 1, 50, default_value=1)
    max_delta_step  = UniformIntegerHyperparameter("max_delta_step ", 0, 50, default_value=0)
    subsample = UniformFloatHyperparameter("subsample", 0.0, 1.0, default_value=1.0)
    cs1.add_hyperparameters([max_depth, learning_rate, n_estimators, gamma, min_child_weight, max_delta_step, subsample])
    return cs1
  return NotImplementedError()

cs = ConfigurationSpace()
for node_idx, n_ in enumerate(final.steps):
  node_name, node = n_
  #print(node_name,"  ", node)
  cs.add_configuration_space(node_name, get_parameter_of_pipeline(node_name))
  print(cs)
  


Configuration space object:
  Hyperparameters:
    normalize:norm, Type: Categorical, Choices: {l1, l2, max}, Default: l2

Configuration space object:
  Hyperparameters:
    Xgboost:gamma, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.0
    Xgboost:learning_rate, Type: UniformFloat, Range: [0.001, 1.0], Default: 0.1
    Xgboost:max_delta_step , Type: UniformInteger, Range: [0, 50], Default: 0
    Xgboost:max_depth, Type: UniformInteger, Range: [1, 10], Default: 3
    Xgboost:min_child_weight, Type: UniformInteger, Range: [1, 50], Default: 1
    Xgboost:n_estimators, Type: UniformInteger, Range: [10, 500], Default: 100
    Xgboost:subsample, Type: UniformFloat, Range: [0.0, 1.0], Default: 1.0
    normalize:norm, Type: Categorical, Choices: {l1, l2, max}, Default: l2



In [0]:
def model_with_pre(cfg):
    print(cfg)
    return 1  # Minimize!
  
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 5,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })

# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
        tae_runner=model_with_pre)
start = time.time()
incumbent = smac.optimize()
inc_value = model_with_pre(incumbent)
end = time.time()
print("Optimized Value: %.5f" % (1-inc_value))  
print("Best parameter : ", incumbent.get_dictionary())
print("Time required in seconds :", (end - start))

Optimizing! Depending on your machine, this might take a few minutes.
Configuration:
  Xgboost:gamma, Value: 0.0
  Xgboost:learning_rate, Value: 0.1
  Xgboost:max_delta_step , Value: 0
  Xgboost:max_depth, Value: 3
  Xgboost:min_child_weight, Value: 1
  Xgboost:n_estimators, Value: 100
  Xgboost:subsample, Value: 1.0
  normalize:norm, Value: 'l2'

Configuration:
  Xgboost:gamma, Value: 0.8226707800302071
  Xgboost:learning_rate, Value: 0.735653908700571
  Xgboost:max_delta_step , Value: 38
  Xgboost:max_depth, Value: 8
  Xgboost:min_child_weight, Value: 37
  Xgboost:n_estimators, Value: 301
  Xgboost:subsample, Value: 0.7415328932596199
  normalize:norm, Value: 'l1'

Configuration:
  Xgboost:gamma, Value: 0.4044836092727059
  Xgboost:learning_rate, Value: 0.6389708978158335
  Xgboost:max_delta_step , Value: 36
  Xgboost:max_depth, Value: 6
  Xgboost:min_child_weight, Value: 20
  Xgboost:n_estimators, Value: 288
  Xgboost:subsample, Value: 0.5903479012826267
  normalize:norm, Value: 'l1

In [0]:
class Person: 
  def __init__(self, age):
    self.age = age
  def get_age(age):
    return age
    
p = Person(56)
print(p.age)


56


In [23]:
final.steps[0][1].set_params(copy=False)

Normalizer(copy=False, norm='l2')